## 13-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

In [1]:
! mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
! ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

ln: failed to create symbolic link '/aiffel/aiffel/recommendata_iu/data/ml-1m/movies.dat': File exists
ln: failed to create symbolic link '/aiffel/aiffel/recommendata_iu/data/ml-1m/ratings.dat': File exists
ln: failed to create symbolic link '/aiffel/aiffel/recommendata_iu/data/ml-1m/README': File exists
ln: failed to create symbolic link '/aiffel/aiffel/recommendata_iu/data/ml-1m/users.dat': File exists


In [29]:
import numpy as np
import scipy
import implicit
import pandas as pd
import datetime as dt

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


### 1) 데이터 준비와 전처리

----------------------------

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [86]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [87]:
ratings.drop(columns = "timestamp", inplace = True)

In [88]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [89]:
# ratings 데이터 확인
ratings.head()

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [90]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [91]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

### 2) 분석해 봅시다.

-----------------------------

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [92]:
# 유니크한 영화개수 유니크한 사용자 수
print("ratings 데이터에 있는 유니크한 영화개수 : ",ratings["movie_id"].nunique())
print("ratings 데이터에 있는 유니크한 영화개수 : ", ratings["user_id"].nunique())

ratings 데이터에 있는 유니크한 영화개수 :  3628
ratings 데이터에 있는 유니크한 영화개수 :  6039


In [93]:
temp = ratings.groupby(["movie_id"])[["counts"]].sum()
a = temp.sort_values(ascending = False, by = "counts").head(30)

# top 30 영화
b = [i for i in a.index]
movies.iloc[b]

,movie_id,title,genre
2858,2927,Brief Encounter (1946),Drama|Romance
260,263,Ladybird Ladybird (1994),Drama
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
2028,2097,Something Wicked This Way Comes (1983),Children's|Horror
1210,1228,Raging Bull (1980),Drama
1198,1216,"Big Blue, The (Le Grand Bleu) (1988)",Adventure|Romance
593,597,Pretty Woman (1990),Comedy|Romance
2571,2640,Superman (1978),Action|Adventure|Sci-Fi
2762,2831,"Dog of Flanders, A (1999)",Drama
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller


### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [94]:
# 영화 찾기
movies[movies["title"].str.lower().str.contains("lion")]

,movie_id,title,genre
360,364,"Lion King, The (1994)",Animation|Children's|Musical
1980,2049,"Happiest Millionaire, The (1967)",Comedy|Musical


In [95]:
# 내가 좋아하는 영화
my_favorite = ["Toy Story (1995)", "Jumanji (1995)", "Amadeus (1984)", "When Harry Met Sally... (1989)", "Lion King, The (1994)"]

In [96]:
movies[movies["title"] == "Lion King, The (1994)"].iloc[0][0]

364

In [97]:
# 좋아하는 영화 movie_id 추출
my_fav_idx = []
for i in my_favorite:
    my_fav_idx.append(movies[movies["title"] == i].iloc[0][0])
my_fav_idx

[1, 2, 1225, 1307, 364]

In [98]:
# 내가 좋아하는 영화
my_favorite = ["Toy Story (1995)", "Jumanji (1995)", "Amadeus (1984)", "When Harry Met Sally... (1989)", "Lion King, The (1994)"]

my_movies = pd.DataFrame({"user_id" : ["Taejong"]*5, "movie_id": my_fav_idx, "counts" : [5, 4, 4, 5, 4]})

if not ratings.isin({"user_id" : ["Taejong"]})["user_id"].any():
    ratings = ratings.append(my_movies)

ratings.tail(10)

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,Taejong,1,5
1,Taejong,2,4
2,Taejong,1225,4
3,Taejong,1307,5
4,Taejong,364,4


In [99]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique= ratings["user_id"].unique()
movies_unique = movies["title"].unique()

# 유저, 영화 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movies_unique)}

In [100]:
movie_to_idx["When Harry Met Sally... (1989)"]  # 잘 작동한다.

1287

In [101]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# # movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
# temp_movie_data = movies['title'].map(movie_to_idx.get).dropna()
# if len(temp_movie_data) == len(movies):
#     print('movie column indexing OK!!')
#     ratings['movie_id'] = temp_movie_data
# else:
#     print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!


,user_id,movie_id,counts
0,0,1193,5
1,0,661,3
2,0,914,3
3,0,3408,4
4,0,2355,5
...,...,...,...
0,6039,1,5
1,6039,2,4
2,6039,1225,4
3,6039,1307,5


In [19]:
# 내가 만든 유저 인덱싱화 함수
# def usertoidx(x):
#     idx = user_to_idx[x]
#     return idx

# ratings["user_id"] = ratings["user_id"].apply(usertoidx)

### 4) CSR matrix를 직접 만들어 봅시다.

In [102]:
print("ratings.user_id.shape : ",ratings.user_id.shape)
print("ratings.movie_id.shape : ", ratings.movie_id.shape)

ratings.user_id.shape :  (836483,)
ratings.movie_id.shape :  (836483,)


In [104]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

ValueError: column index exceeds matrix dimensions

In [24]:
ratings.counts

0    5
1    3
2    3
3    4
4    5
    ..
0    5
1    4
2    4
3    5
4    4
Name: counts, Length: 836483, dtype: int64

In [25]:
ratings.user_id

0       0
1       0
2       0
3       0
4       0
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user_id, Length: 836483, dtype: int64

In [26]:
ratings.movie_id

0    1193
1     661
2     914
3    3408
4    2355
     ... 
0       1
1       2
2    1225
3    1307
4     364
Name: movie_id, Length: 836483, dtype: int64

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.